<a href="https://colab.research.google.com/github/1eetnals/Child-abuse-detection-system-using-TEN4FACESp/blob/main/image_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 깨져있는 데이터, 해상도 설정해줘서 기준 미달하면 제거

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import time
import socket


from urllib.request import urlretrieve
from urllib.error import HTTPError, URLError
#from selenium import webdriver
#from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, \
    #ElementNotInteractableException
from PIL import Image
#from pygame import mixer

path = '/content/drive/Shareddrives/BME캡스톤/data/practice_none'
date = "arm"
def filtering():
    print("ㅡ 필터링 시작 ㅡ")
    filtered_count = 0
    dir_name = f"{path}/{date}"
    for index, file_name in enumerate(os.listdir(dir_name)):
        try:
            file_path = os.path.join(dir_name, file_name)
            img = Image.open(file_path)

            # 이미지 해상도의 가로와 세로가 모두 350이하인 경우
            if img.width < 351 and img.height < 351:
                img.close()
                os.remove(file_path)
                print(f"{index} 번째 사진 삭제")
                filtered_count += 1

        # 이미지 파일이 깨져있는 경우
        except OSError:
            os.remove(file_path)
            filtered_count += 1

    print(f"ㅡ 필터링 종료 (총 갯수: {227 - filtered_count}) ㅡ")

In [ ]:
filtering()

ㅡ 필터링 시작 ㅡ
ㅡ 필터링 종료 (총 갯수: 227) ㅡ


## 2. 같은 이미지 확인 코드

In [ ]:
import os
photo_list = []

for f in os.listdir('/content/drive/Shareddrives/BME캡스톤/data/strawberry'):
    if 'jpg' in f:
        photo_list.append(f)

In [ ]:
photo_list

['20220213000003506 (1).jpg',
 'dsjkeksn.jpg',
 'images.jpg',
 '99FEB03C5FCE37911C.jpg',
 'images (1).jpg',
 '다운로드.jpg',
 'download.jpg']

In [ ]:
photo_size = list(map(lambda x: os.path.getsize('/content/drive/Shareddrives/BME캡스톤/data/strawberry' + '/' + x), photo_list))

In [ ]:
photo_size

[90555, 90555, 3863, 148498, 9401, 7140, 7140]

In [ ]:
# Find Same Photos
import pandas as pd
fsp = pd.DataFrame({'filename_raw':photo_list, 'size':photo_size})

print('사진의 갯수 :', len(fsp))

사진의 갯수 : 7


In [ ]:
import re   # 정규표현식
com = re.compile(' \d')
fsp['filename'] = list(map(lambda x: com.sub('', x), photo_list))

In [ ]:
fsp

,filename_raw,size,filename
0,20220213000003506 (1).jpg,90555,20220213000003506 (1).jpg
1,dsjkeksn.jpg,90555,dsjkeksn.jpg
2,images.jpg,3863,images.jpg
3,99FEB03C5FCE37911C.jpg,148498,99FEB03C5FCE37911C.jpg
4,images (1).jpg,9401,images (1).jpg
5,다운로드.jpg,7140,다운로드.jpg
6,download.jpg,7140,download.jpg


In [ ]:
# Photo Value Counts
pvc = pd.DataFrame({'filename':fsp['filename'].value_counts().index, 'fn_counts':fsp['filename'].value_counts().values})   
psvc = pd.DataFrame({'size':fsp['size'].value_counts().index, 'size_counts':fsp['size'].value_counts().values})   

fsp = pd.merge(fsp, pvc, how = 'left', on = 'filename')
fsp = pd.merge(fsp, psvc, how = 'left', on = 'size')

fsp.sample(2)

,filename_raw,size,filename,fn_counts,size_counts
6,download.jpg,7140,download.jpg,1,2
5,다운로드.jpg,7140,다운로드.jpg,1,2


In [ ]:
for i in range(len(fsp)):
    if (fsp['fn_counts'][i] > 1) & (fsp['size_counts'][i] == 1):
        print(i)

In [ ]:
# Find Same Phto_Not Same Name
fsp_nsn = fsp.sort_values(['filename_raw'], ascending = False).drop_duplicates(['filename'], keep = 'first')

In [ ]:
print('남은 사진의 갯수 : {}\n지워진 사진의 갯수 : {}'.format(len(fsp_nsn), len(fsp)-len(fsp_nsn)))

남은 사진의 갯수 : 7
지워진 사진의 갯수 : 0


In [ ]:
pvc_nsn = pd.DataFrame({'filename':fsp_nsn['filename'].value_counts().index, 'fn_counts_nsn':fsp_nsn['filename'].value_counts().values})   
psvc_nsn = pd.DataFrame({'size':fsp_nsn['size'].value_counts().index, 'size_counts_nsn':fsp_nsn['size'].value_counts().values})   

fsp_nsn = pd.merge(fsp_nsn, pvc_nsn, how = 'left', on = 'filename')
fsp_nsn = pd.merge(fsp_nsn, psvc_nsn, how = 'left', on = 'size')

In [ ]:
fsp_nsn[fsp_nsn['fn_counts_nsn']!=1]

,filename_raw,size,filename,fn_counts,size_counts,fn_counts_nsn,size_counts_nsn


In [ ]:
print('사이즈 겹치는 사진의 갯수 :', len(fsp_nsn[fsp_nsn['size_counts_nsn']!=1]))
print('중복 사이즈의 갯수 :', len(psvc_nsn[psvc_nsn['size_counts_nsn']>1]))

사이즈 겹치는 사진의 갯수 : 4
중복 사이즈의 갯수 : 2


In [ ]:
print(grayA.shape)
print(grayB.shape)
print(temp['filename_raw'][0])
print(temp['filename_raw'][1])
print(grayA[200])
print(grayB[200])
(grayA-grayB)[200]

(400, 690)
(400, 690)
dsjkeksn.jpg
20220213000003506 (1).jpg
[255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 25

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [ ]:
import cv2   # OpenCV
#from skimage.measure import compare_ssim
from skimage.metrics import structural_similarity as ssim

# 삭제될 사진의 리스트
delete = []


for i in range(len(psvc_nsn)):
    
    # 중복된 크기(size)가 있는 경우
    if psvc_nsn['size_counts_nsn'][i] == 2:
        
        # 그 크기에 해당하는 사진을 불러온다. 
        temp = fsp_nsn[fsp_nsn['size']==psvc_nsn['size'][i]].reset_index(drop = True).sort_values(['filename'])
        
        # 사진 읽기
        imageA = cv2.imread('/content/drive/Shareddrives/BME캡스톤/data/strawberry/'+temp['filename_raw'][0])
        imageB = cv2.imread('/content/drive/Shareddrives/BME캡스톤/data/strawberry/'+temp['filename_raw'][1])
        
        # 이미지를 grayscale로 변환
        grayA = cv2.cvtColor(imageA,cv2.COLOR_RGB2GRAY)
        grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)
        
        # 이미지의 구조가 같다면 이미지 비교
        if len(grayA)==len(grayB):
            (score, diff) = ssim(grayA, grayB, full=True)
            
            # 차이가 없다면 하나는 delete에 넣어주기
            if score == 1:
                delete.append(temp['filename_raw'][1])
            
            # 구조가 같지만 차이가 존재한다면 직접 확인하기     
            else:
                print('확인해보시오! : ', temp['filename_raw'][0], '/', temp['filename_raw'][1], f'(score : {score})')

확인해보시오! :  다운로드.jpg / download.jpg (score : 0.977819595477938)
확인해보시오! :  dsjkeksn.jpg / 20220213000003506 (1).jpg (score : 0.9871611578188457)


In [ ]:
# 중복제거된 것들은 delete 리스트에 넣어주기
delete = delete + list(fsp[~fsp['filename_raw'].isin(fsp_nsn['filename_raw'])]['filename_raw'])

print('삭제할 목록 :', delete)

삭제할 목록 : []


아래에껀 실행X


In [ ]:
# result : 처음(fsp)데이터에서 - delete를 제외한 것
result = list(fsp[~fsp['filename_raw'].isin(delete)]['filename_raw'])

print('남길 목록 : ', result)

남길 목록 :  ['leg116.jpg', 'leg114.jpg', 'leg123.jpg', 'leg115.jpg', 'leg113.jpg', 'leg112.jpg', 'leg111.jpg', 'leg109.jpg', 'leg11.jpg', 'leg108.jpg', 'leg110.jpg', 'leg106.jpg', 'leg105.jpg', 'leg107.jpg', 'leg100.jpg', 'leg103.jpg', 'leg101.jpg', 'leg104.jpg', 'leg102.jpg', 'leg10.jpg', 'leg2.jpg', 'leg20.jpg', 'leg153.jpg', 'leg151.jpg', 'leg150.jpg', 'leg15.jpg', 'leg149.jpg', 'leg146.jpg', 'leg148.jpg', 'leg147.jpg', 'leg154.jpg', 'leg145.jpg', 'leg152.jpg', 'leg144.jpg', 'leg135.jpg', 'leg143.jpg', 'leg136.jpg', 'leg141.jpg', 'leg138.jpg', 'leg142.jpg', 'leg139.jpg', 'leg14.jpg', 'leg137.jpg', 'leg140.jpg', 'leg134.jpg', 'leg126.jpg', 'leg133.jpg', 'leg130.jpg', 'leg131.jpg', 'leg13.jpg', 'leg132.jpg', 'leg129.jpg', 'leg128.jpg', 'leg127.jpg', 'leg125.jpg', 'leg124.jpg', 'leg120.jpg', 'leg119.jpg', 'leg121.jpg', 'leg12.jpg', 'leg118.jpg', 'leg117.jpg', 'leg122.jpg', 'leg5.jpg', 'leg51.jpg', 'leg57.jpg', 'leg49.jpg', 'leg46.jpg', 'leg48.jpg', 'leg44.jpg', 'leg47.jpg', 'leg45.jpg', '

In [ ]:
import shutil

for i in result:
    shutil.move('/content/drive/Shareddrives/BME캡스톤/data/practice_none/leg/'+i, '/content/drive/Shareddrives/BME캡스톤/data/practice_none/leg/Result')
    
for i in delete:
    shutil.move('/content/drive/Shareddrives/BME캡스톤/data/practice_none/leg/'+i, '/content/drive/Shareddrives/BME캡스톤/data/practice_none/leg/Delete')   